In [1]:
#import designs_setup as ds
import numpy as np
from setup_functions import *

In [9]:
myList = [[1,2,3], [2,2], [1], [1,1,1,1], [], [2,2],[0,0,0]]
x = {len(i) for i in myList}
y = [len(i) for i in myList]
print(x)
print(y)

# Other methods

In [2]:
####### Construct Network ########

N = 15**2        # population size
n = 15
NC = 5**2          # number of clusters
nc = 5
print(N,n,NC,nc)
diag = 1        # maximum norm of direct effect
r = 1.25        # ratio between indirect and direct effects
p = 0.05        # treatment probability

# Create lattice network
A = lattice2Dsq(n,n);

225 15 25 5


/home/mayleencortez/networkcausal2022/lib/python3.8/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [3]:
clusters, clusters_flat = bf_clusters(NC,N)
print(clusters)

[[ 0  0  0  1  1  1  2  2  2  3  3  3  4  4  4]
 [ 0  0  0  1  1  1  2  2  2  3  3  3  4  4  4]
 [ 0  0  0  1  1  1  2  2  2  3  3  3  4  4  4]
 [ 5  5  5  6  6  6  7  7  7  8  8  8  9  9  9]
 [ 5  5  5  6  6  6  7  7  7  8  8  8  9  9  9]
 [ 5  5  5  6  6  6  7  7  7  8  8  8  9  9  9]
 [10 10 10 11 11 11 12 12 12 13 13 13 14 14 14]
 [10 10 10 11 11 11 12 12 12 13 13 13 14 14 14]
 [10 10 10 11 11 11 12 12 12 13 13 13 14 14 14]
 [15 15 15 16 16 16 17 17 17 18 18 18 19 19 19]
 [15 15 15 16 16 16 17 17 17 18 18 18 19 19 19]
 [15 15 15 16 16 16 17 17 17 18 18 18 19 19 19]
 [20 20 20 21 21 21 22 22 22 23 23 23 24 24 24]
 [20 20 20 21 21 21 22 22 22 23 23 23 24 24 24]
 [20 20 20 21 21 21 22 22 22 23 23 23 24 24 24]]


In [17]:
i = 44
print(A[i,:])

  (0, 14)	1
  (0, 43)	1
  (0, 44)	1
  (0, 45)	1
  (0, 74)	1


In [18]:
k = int(np.ceil(n/nc))
print(k)

3


In [19]:
lst = cluster_neighborhood(A,i,k)
print(lst)

[(0, 4), (0, 5)]


In [21]:
p = 0.5
Cz, flatCz, z = bernoulli_cluster(n,p,clusters)
for l in lst:
    print(l)
    print((Cz[l]/p) - (1-Cz[l])/(1-p))

(0, 4)
2.0
(0, 5)


IndexError: index 5 is out of bounds for axis 1 with size 5

# Estimators

In [4]:
def clusterPI(A,Y,Cz,p):
    '''
    Returns an estimate of the TTE using the (beta=1) cluster pseudoinverse estimator.

    Parameters
    ----------
    A : scipy sparse array
        represents N x N adjacency matrix for a population of size N = n^2
    Y : numpy array
        N x 1 array of observed outcomes, one for each unit in the population
    Cz : numpy array
        nc by nc array of cluster treatement assignments (nc^2 is the number of clusters)
        (s,t)-th element is treatment assignment of cluster (s,t) in [nc] x [nc]
    p : float
        treatment probability of any cluster

    Returns
    -------
    TTE_hat/N : float
        estimate of the total treatment effect
    '''
    N = np.shape(A)[0]     # population size
    nc = np.shape(Cz)[0]   # square root of the number of clusters
    n = int(np.sqrt(N))    # square root of population size
    k = int(np.ceil(n/nc)) # "typical" cluster side length
    
    TTE_hat = 0
    for i in range(N):
        #print(i)
        sum = 0
        for c in cluster_neighborhood(A,i,k):
            sum = sum + (Cz[c]/p) - (1-Cz[c])/(1-p)
        TTE_hat = TTE_hat + Y[i]*sum
    return TTE_hat/N

In [17]:
def ht_weights_bernoulli(N,p,neighborhoods):
    '''
    Returns the inverse probability weights for Horvitz-Thompson estimator under Bernoulli(p) cluster design

    Parameters
    ----------
    N : int
        number of units in the population (n^2)
    p : float
        cluster treatment probability
    neighborhoods: list
        each entry i contains a list of clusters adjacent to i, i.e. the cluster neighborhood of i

    Returns
    --------
    w1 : 1D numpy array
        w1[i] = 1/Prob(i's neighborhood fully assigned to treatment)
    w0 : 1D numpy array
        w0[i] = 1/Prob(i's neighborhood fully assigned to control); inverse prob weight
    '''
    w1 = np.zeros(N)    # w1[i] = 1/Prob(i's neighborhood fully assigned to treatment)
    w0 = np.zeros(N)    # w0[i] = 1/Prob(i's neighborhood fully assigned to control)
    K = [len(i) for i in neighborhoods] # K[i] = size of i's cluster neighborhood
    w1 = 1/p**K
    w0 = 1/(1-p)**K

    return w1,w0

In [16]:
def ht(p,z,w1,w0,A,Y):
    '''
    Returns the Horvitz-Thompson estimate of the TTE

    Parameters
    ----------
    p : float
        treatment probability of clusters in Bernoulli(p) cluster design
    z : 1D numpy array
        z[i] = treatment assignment of unit i in a population of size n^2
    w1: 1D numpy array
        w1[i] = 1/Prob(i's neighborhood fully assigned to treatment); inverse prob weight
    w0: 1D numpy array
        w0[i] = 1/Prob(i's neighborhood fully assigned to control); inverse prob weight
    A : scipy sparse array
        represents N x N adjacency matrix for a population of size N = n^2
    Y : 1D numpy array
        Y[i] = observed outcome of unit i in a population of size n^2
    
    Returns
    -------
    est : float
        Horvitz-Thompson estimate of the TTE
    '''
    N = np.shape(A)[0]  # population size
    est=0

    Ind1 = np.zeros(N)  # Ind1[i] = Indicator(i's neighborhood fully assigned to treatment)
    Ind0 = np.zeros(N)  # Ind0[i] = Indicator(i's neighborhood fully assigned to control)
    Ind1 = (A.dot(z) == A.sum(axis=1)) + 0
    Ind0 = (A.dot(z) == np.zeroes(N)) + 0

    est = (1/N) * np.sum(Y.dot(Ind1/w1 - Ind0/w0))
    return est

Experiment to test the unbiasedness of the estimator

In [5]:
####### Construct Network ########

N = 900        # population size
n = int(np.sqrt(N))
NC = 100          # number of clusters
nc = int(np.sqrt(NC))
#print(N,n,NC,nc)
diag = 1        # maximum norm of direct effect
r = 1.25        # ratio between indirect and direct effects
p = 0.05        # treatment probability

# Create lattice network
A = lattice2Dsq(n,n)

# Generate edge weights
rand_wts = np.random.rand(N,3)
alpha = rand_wts[:,0].flatten()
C = simpleWeights(A, diag, r*diag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())

In [6]:
# Potential Outcomes Model
fy = lambda z: linear_pom(C,alpha,z)

# Calculate and print ground truth TTE
TTE = 1/N * np.sum((fy(np.ones(N)) - fy(np.zeros(N))))
print("Ground-Truth TTE: {}\n".format(TTE))

Ground-Truth TTE: 1.1391062385491877



In [7]:
_, clusters = bf_clusters(NC,N)

In [9]:
####### Estimate ########

p = 0.5    # treatment probability
T = 500   # number of trials

#TTE_PI, TTE_ht, TTE_hajek = np.zeros(T), np.zeros(T), np.zeros(T)
TTE_PI= np.zeros(T)

In [10]:
for i in range(T):
  Cz, flatCz, z = bernoulli_cluster(n,p,clusters)
  Y = fy(z)
  #print(i)
  TTE_PI[i] = clusterPI(A,Y,Cz,p)
  #TTE_ht[i] = ht(n,p,Y,A,z)
  #TTE_hajek[i] = hajek(n,p,y,A,z)

In [11]:
print("Ours: {}".format(np.sum(TTE_PI)/T))
print("Our bias: {}".format(TTE-(np.sum(TTE_PI)/T)))
print("Our MSE: {}\n".format(np.sum((TTE_PI-TTE)**2)/T))

# print("H-T: {}".format(np.sum(TTE_ht)/T))
# print("H-T MSE: {}\n".format(np.sum((TTE_ht-TTE)**2)/T))

# print("Hajek: {}".format(np.sum(TTE_hajek)/T))
# print("Hajek MSE: {}".format(np.sum((TTE_hajek-TTE)**2)/T))

Ours: 0.013716418068646223
Our bias: 1.1253898204805415
Our MSE: 1.4557261130085148



### When $k$ divides $n$
First, I'm testing whether my code works when $k | n$. I'm making the choice that when $k | n$ we don't sample a random origin. 

In [11]:
n = 12
k = 3

print('Cluster',':','I ',' J')
for i in range(n//k):
    for j in range(n//k):
        I,J = ds.sqlat_toUnit(i,j,k,n)
        print((i,j),':', I,J)
    print()

Cluster : I   J
(0, 0) : [0 1 2] [0 1 2]
(0, 1) : [0 1 2] [3 4 5]
(0, 2) : [0 1 2] [6 7 8]
(0, 3) : [0 1 2] [ 9 10 11]

(1, 0) : [3 4 5] [0 1 2]
(1, 1) : [3 4 5] [3 4 5]
(1, 2) : [3 4 5] [6 7 8]
(1, 3) : [3 4 5] [ 9 10 11]

(2, 0) : [6 7 8] [0 1 2]
(2, 1) : [6 7 8] [3 4 5]
(2, 2) : [6 7 8] [6 7 8]
(2, 3) : [6 7 8] [ 9 10 11]

(3, 0) : [ 9 10 11] [0 1 2]
(3, 1) : [ 9 10 11] [3 4 5]
(3, 2) : [ 9 10 11] [6 7 8]
(3, 3) : [ 9 10 11] [ 9 10 11]



In [13]:
I = np.array([1,2])
J = np.array([1,2])
s,t = ds.sqlat_toCluster(I,J,k,divides=True)
print(s,t)

[0 0] [0 0]


In [3]:
print('Unit', ':', 'Cluster')
for i in [5,10]:
    for j in range(n):
        s,t = ds.sqlat_toCluster(i,j,k,divides=True)
        print((i,j),':',(s,t))
    print()

Unit : Cluster
(5, 0) : (1, 0)
(5, 1) : (1, 0)
(5, 2) : (1, 0)
(5, 3) : (1, 1)
(5, 4) : (1, 1)
(5, 5) : (1, 1)
(5, 6) : (1, 2)
(5, 7) : (1, 2)
(5, 8) : (1, 2)
(5, 9) : (1, 3)
(5, 10) : (1, 3)
(5, 11) : (1, 3)

(10, 0) : (3, 0)
(10, 1) : (3, 0)
(10, 2) : (3, 0)
(10, 3) : (3, 1)
(10, 4) : (3, 1)
(10, 5) : (3, 1)
(10, 6) : (3, 2)
(10, 7) : (3, 2)
(10, 8) : (3, 2)
(10, 9) : (3, 3)
(10, 10) : (3, 3)
(10, 11) : (3, 3)



In [4]:
k = 4

print('Cluster',':','I ',' J')
for i in range(n//k):
    for j in range(n//k):
        I,J = ds.sqlat_toUnit(i,j,k,n)
        print((i,j),':', I,J)
    print()

Cluster : I   J
(0, 0) : [0 1 2 3] [0 1 2 3]
(0, 1) : [0 1 2 3] [4 5 6 7]
(0, 2) : [0 1 2 3] [ 8  9 10 11]

(1, 0) : [4 5 6 7] [0 1 2 3]
(1, 1) : [4 5 6 7] [4 5 6 7]
(1, 2) : [4 5 6 7] [ 8  9 10 11]

(2, 0) : [ 8  9 10 11] [0 1 2 3]
(2, 1) : [ 8  9 10 11] [4 5 6 7]
(2, 2) : [ 8  9 10 11] [ 8  9 10 11]



In [5]:
print('Unit', ':', 'Cluster')
for i in [3,9]:
    for j in range(n):
        s,t = ds.sqlat_toCluster(i,j,k,divides=True)
        print((i,j),':',(s,t))
    print()

Unit : Cluster
(3, 0) : (0, 0)
(3, 1) : (0, 0)
(3, 2) : (0, 0)
(3, 3) : (0, 0)
(3, 4) : (0, 1)
(3, 5) : (0, 1)
(3, 6) : (0, 1)
(3, 7) : (0, 1)
(3, 8) : (0, 2)
(3, 9) : (0, 2)
(3, 10) : (0, 2)
(3, 11) : (0, 2)

(9, 0) : (2, 0)
(9, 1) : (2, 0)
(9, 2) : (2, 0)
(9, 3) : (2, 0)
(9, 4) : (2, 1)
(9, 5) : (2, 1)
(9, 6) : (2, 1)
(9, 7) : (2, 1)
(9, 8) : (2, 2)
(9, 9) : (2, 2)
(9, 10) : (2, 2)
(9, 11) : (2, 2)



### When $k$ doesn't divide $n$
Now I want to test what happens when $k$ does not divide $n$ and hence we sample an origin point $(q_1,q_2).$

In [6]:
from numpy.random import default_rng
n = 7
k = 2
print(n%k == 0)

False


In [7]:
# sample origin
rng = default_rng()
q1 = rng.integers(low=0, high=k)
q2 = rng.integers(low=0, high=k)
print(q1,q2)

1 0


In [8]:
num = np.ceil(n/k)
print('Cluster',':','I  ','  J')
for i in range(num.astype(int)):
    for j in range(num.astype(int)):
        I,J = ds.sqlat_toUnit(i,j,k,n,q1,q2)
        print((i,j),':', I,J)
    print()

Cluster : I     J
(0, 0) : [0 1] [0]
(0, 1) : [0 1] [1 2]
(0, 2) : [0 1] [3 4]
(0, 3) : [0 1] [5 6]

(1, 0) : [2 3] [0]
(1, 1) : [2 3] [1 2]
(1, 2) : [2 3] [3 4]
(1, 3) : [2 3] [5 6]

(2, 0) : [4 5] [0]
(2, 1) : [4 5] [1 2]
(2, 2) : [4 5] [3 4]
(2, 3) : [4 5] [5 6]

(3, 0) : [6] [0]
(3, 1) : [6] [1 2]
(3, 2) : [6] [3 4]
(3, 3) : [6] [5 6]



In [9]:
q1 = rng.integers(low=0, high=k)
q2 = rng.integers(low=0, high=k)
print(q1,q2)

0 0


In [10]:
num = np.ceil(n/k)
print('Cluster',':','I  ','  J')
for i in range(num.astype(int)):
    for j in range(num.astype(int)):
        I,J = ds.sqlat_toUnit(i,j,k,n,q1,q2)
        print((i,j),':', I,J)
    print()

Cluster : I     J
(0, 0) : [0] [0]
(0, 1) : [0] [1 2]
(0, 2) : [0] [3 4]
(0, 3) : [0] [5 6]

(1, 0) : [1 2] [0]
(1, 1) : [1 2] [1 2]
(1, 2) : [1 2] [3 4]
(1, 3) : [1 2] [5 6]

(2, 0) : [3 4] [0]
(2, 1) : [3 4] [1 2]
(2, 2) : [3 4] [3 4]
(2, 3) : [3 4] [5 6]

(3, 0) : [5 6] [0]
(3, 1) : [5 6] [1 2]
(3, 2) : [5 6] [3 4]
(3, 3) : [5 6] [5 6]

